In [1]:
# this is still a very early version, and still working on making a framework that will get basic things going. Will enhance and polish all areas afterwards.
# todo:
#   add more features
#   implement bayesian optimization

In [2]:
# import libraries
from marketstackAPI import Marketstack
import pandas as pd
import numpy as np
import cufflinks as cf
import ta
import holidays
import matplotlib as plt
import plotly.graph_objects as go
import lightgbm as lgb

from datetime import datetime
from datetime import timedelta
from imblearn.over_sampling import SMOTE
from IPython.core.display import display, HTML

In [3]:
# jupyter notebook settings and chart size configs
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['figure.dpi'] = 200
pd.options.plotting.backend = "plotly"

In [4]:
# initialize and set parameters
MS = Marketstack() # requires API key from Marketstack with basic plan to get 10 years worth of data
cf.set_config_file(theme='henanigans',sharing='public',offline=True)

In [5]:
def raw_data_preprocessing(raw_data):
    """
    Clean raw_data by removing extra columns, renaming columns, order by date in descending order, reset index number.
    this data format will be used as the standard format for all other feature engineering related function calls.
    
    Parameters
    ----------
    raw_data : pandas dataframe that contains ['date','adj_high','adj_low','adj_close','adj_open','adj_volume'] columns, ordered by date in ascending order.
    
    Return:
    ----------
    standard_data: pandas dataframe that contains ['date','high','low','close','open','volume'] columns, ordered by date in descending order.
    
    """
    data = raw_data[:]
    data = data[['date','adj_high','adj_low','adj_close','adj_open','adj_volume']]
    data.columns = ['date','high','low','close','open','volume']
    data = data[::-1]
    data.reset_index(inplace=True, drop=True)
    return data

In [6]:
def get_ta_indicators(standard_data, prefix = ''):
    """
    Compute technical indicators for every period, each row within standard_data is a period.
    
    Parameters
    ----------
    standard_data : pandas dataframe that contains ['date','high','low','close','open','volume'] columns, ordered by date in descending order.
    prefix: string that will be concatennated to before all technical indicators names
    
    Return:
    ----------
    data: pandas dataframe that contains computed technical indicators with corrsponding name
    
    """
    data = standard_data[:]
    df = pd.DataFrame()
    df.insert(0, prefix+'_stochrsi_14' if prefix else 'stochrsi_14', ta.momentum.stochrsi(close = data.close)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_mfi_14' if prefix else 'mfi_14', ta.volume.money_flow_index(high = data.high, low = data.low, close = data.close, volume= data.volume)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_adx_14' if prefix else 'adx_14', ta.trend.adx(high = data.high, low = data.low, close = data.close)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_adx_neg_14' if prefix else 'adx_neg_14', ta.trend.adx_neg(high = data.high, low = data.low, close = data.close)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_adx_pos_14' if prefix else 'adx_pos_14', ta.trend.adx_pos(high = data.high, low = data.low, close = data.close)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_aroon_up_25' if prefix else 'aroon_up_25', ta.trend.aroon_up(close = data.close)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_aroon_down_25' if prefix else 'aroon_down_25', ta.trend.aroon_down(close = data.close)/100) # range 0 to 100 rescaled to 0 to 1
    df.insert(0, prefix+'_aroon_25' if prefix else 'aroon_25', (ta.trend.aroon_up(close = data.close) - ta.trend.aroon_down(close = data.close))/100) # range 0 to 100 rescaled to 0 to 1
    
    return df

In [7]:
def get_percent_changes(standard_data, prefix = ''):
    """
    Compute basic % changes
    
    Parameters
    ----------
    standard_data : pandas dataframe that contains ['date','high','low','close','open','volume'] columns, ordered by date in descending order.
    prefix: string that will be concatennated to before all technical indicators names
    
    Return:
    ----------
    data: pandas dataframe that contains computed % changes indicators with corrsponding name
    
    """
    data = standard_data[:]
    df = pd.DataFrame()
    #add volume % change from yesterday to today
    df.insert(0,'volume_change',data.volume/data.volume.shift(1)-1)
    #add price % change from yesterday to today
    df.insert(0,'price_change',data.close/data.close.shift(1)-1)
    return df

In [8]:
def get_target_variable(standard_data):
    """
    Compute target variable.
    the target variable indicates three classes.
    2 : next day is going up significantly
    0: next day is going down significantly
    1 : no significant movement for the next day.
    
    How significant change is defined using more than 1% change at the moment. could be changing to something else.
    
    
    Parameters
    ----------
    standard_data : pandas dataframe that contains ['date','high','low','close','open','volume'] columns, ordered by date in descending order.
    
    Return:
    ----------
    data: pandas dataframe that contains target variable
    
    """
    data = standard_data[:]
    #creating Y
    #calculate daily % change using daily close using the NEXT day close / today close
    df = pd.DataFrame()
    target = data.close.shift(-1)/data.close-1
    target[target > 0.01] = 1
    target[target < -0.01] = -1
    target[(target < 1) & (target > -1)] = 0
    target += 1
    df.insert(0,'target', target)
    df.insert(1,'change', data.close.shift(-1)/data.close-1)
    return df

In [9]:
# remove common rows with nan from full_data and target and return new dataset
def remove_nan(full_data, target):
    to_keep = [not x for x in np.array(list(map(any,full_data.isna().values))) | np.array(list(map(any,target.isna().values)))]
    full_data = full_data[to_keep]
    target = target[to_keep]
    return full_data, target

In [10]:
def eval_strategy(results, data, prefix = ''):
    # the evaluation process would be a simulation of trading the stocks at the close.
    #   when the prediction is:
    #    0 : short/sell
    #    1 : unclear thus liquidate and wait for long/short signals
    #    2 : long/buy
    #   when there are consective singnals of buy or sell, the action would be to hold
    # assuming no commission per trade, and orders always fill at the close.
    max_balance = 10000
    eval_data = results.merge(data, left_index=True, right_index=True)
    beginning_balance = [10000]
    beginning_cash = [10000]
    shares_owned = [0]
    ending_balance = [10000]
    ending_cash = [10000]
    draw_down = [0]
    actions = ['liquidate']
    predictions = ['liquidate']
    pred_to_action = {
        0:'short',
        1:'liquidate',
        2:'long'
    }
    for idx, row in eval_data.iterrows():
        beginning_balance.append(ending_balance[-1])
        ending_balance.append(ending_cash[-1]+(shares_owned[-1])*row.close)
        beginning_cash.append(ending_cash[-1])
        ending_cash.append(ending_cash[-1])
        shares_owned.append(shares_owned[-1])
        # if same as previous
        if pred_to_action[row.predictions.argmax()] == predictions[-1]:
            action = 'hold'
        # if liquidate
        elif row.predictions.argmax() == 1:
            ending_cash[-1] += shares_owned[-1] * row.close
            shares_owned[-1] = 0
            action = pred_to_action[row.predictions.argmax()]
        # if long
        elif row.predictions.argmax() == 2:
            ending_cash[-1] += shares_owned[-1] * row.close
            shares_owned[-1] = 0
            shares_owned[-1] += np.floor(ending_cash[-1]/row.close)
            ending_cash[-1] -= np.floor(ending_cash[-1]/row.close) * row.close
            action = pred_to_action[row.predictions.argmax()]
        # if short
        elif row.predictions.argmax() == 0:
            ending_cash[-1] += shares_owned[-1] * row.close
            shares_owned[-1] = 0
            shares_owned[-1] -= np.floor(ending_cash[-1]/row.close)
            ending_cash[-1] -= -np.floor(ending_cash[-1]/row.close) * row.close
            action = pred_to_action[row.predictions.argmax()]
        #calc
        max_balance = max(max_balance,ending_balance[-1])
        draw_down.append(ending_balance[-1]/max_balance - 1)
        actions.append(action)
        predictions.append(pred_to_action[row.predictions.argmax()])
    df = pd.DataFrame()
    df.insert(0, 'beginning_balance', beginning_balance)
    df.insert(1, 'beginning_cash', beginning_cash)
    df.insert(2, 'shares_owned', shares_owned)
    df.insert(3, 'ending_cash', ending_cash)
    df.insert(4, 'ending_balance', ending_balance)
    df.insert(5, 'draw_down', draw_down)
    df.insert(6, 'actions', actions)
    df.columns = [prefix+'_'+x if prefix else x for x in df.columns]
    return df[1:]

In [11]:
def eval_buy_and_hold(results, data, prefix = ''):
    # the evaluation process would be a simulation of trading the stocks at the close and always long/short as much as possible and liquidate the next day
    # assuming no commission per trade, and orders always fill at the close.
    max_balance = 10000
    eval_data = results.merge(data, left_index=True, right_index=True)
    beginning_balance = [10000]
    beginning_cash = [10000]
    shares_owned = [0]
    ending_balance = [10000]
    ending_cash = [10000]
    draw_down = [0]
    actions = ['liquidate']
    
    for idx, row in eval_data.iterrows():
        beginning_balance.append(ending_balance[-1])
        ending_balance.append(ending_cash[-1]+(shares_owned[-1])*row.close)
        beginning_cash.append(ending_cash[-1])
        ending_cash.append(ending_cash[-1])
        shares_owned.append(shares_owned[-1])
        
        #liquidate
        ending_cash[-1] += shares_owned[-1] * row.close
        shares_owned[-1] = 0
        action = 'liquidate'
        # long
        shares_owned[-1] += np.floor(ending_cash[-1]/row.close)
        ending_cash[-1] -= (ending_cash[-1]/row.close) * row.close
        action = 'long'

        #calc
        max_balance = max(max_balance,ending_balance[-1])
        draw_down.append(ending_balance[-1]/max_balance - 1)
        actions.append(action)
    df = pd.DataFrame()
    df.insert(0, 'beginning_balance', beginning_balance)
    df.insert(1, 'beginning_cash', beginning_cash)
    df.insert(2, 'shares_owned', shares_owned)
    df.insert(3, 'ending_cash', ending_cash)
    df.insert(4, 'ending_balance', ending_balance)
    df.insert(5, 'draw_down', draw_down)
    df.insert(6, 'actions', actions)
    df.columns = [prefix+'_'+x if prefix else x for x in df.columns]
    return df[1:]

In [12]:
# Since this is a timeseries dataset and because of my personal trading experience I'm claming/assuming the underlying relationship between features and target variables isn't stationary, 
# thus the traning method will be in walk-forward style instead of cross-validation; and for each iteration the model isn't going to using everydata available since they aren't as relevant.
# will try to play with weights in the future to see if applying less weights to "outdated" data will help the model.

def train_and_eval(full_data, target):
    training_window_size = 1500 # days of data to train the model for each iteration. using about 3 years of data
    validation_window_size = 100
    predict_window_size  = 1  # days of data used to test and eval the model for each iteration. Using about 2 weeks of data 
                               # ideally the predict_window_size should be set to 1, but that would take too long to train.
                               # maybe I would try it when I have a good model with good parameters
    best_iteration = 1
    # check full_data len is more than training_window_size + predict_window_size if false throw error
    assert len(full_data) > training_window_size + predict_window_size, "full_data lenght is less than training_window_size + predict_window_size"
    predictions = []
    truths = []
    prediction_results = target[training_window_size:]
    for i in range(training_window_size,len(full_data),predict_window_size):
        validation_size = validation_window_size
        # setup train and test data
        train_x = full_data[i-training_window_size:i-validation_window_size]
        valid_x = full_data[i-validation_window_size:i]
        test_x  = full_data[i:i+predict_window_size]
        
        train_y = target[i-training_window_size:i-validation_window_size]
        valid_y = target[i-validation_window_size:i]
        test_y  = target[i:i+predict_window_size]
        
        # oversample trainning data to balance the dataset
        oversample = SMOTE(k_neighbors = 5, random_state = 0) 
        train_x, train_y = oversample.fit_resample(train_x, train_y.target)
        validation_set_skip = False
        try:
            valid_x, valid_y = oversample.fit_resample(valid_x, valid_y.target)
            validation_data = lgb.Dataset(valid_x, label=valid_y, reference=train_data, free_raw_data=False)
        except:
            validation_set_skip = True # not enough samples vs neighbors, skipping validation for this iteration and keep using last iteration model parameters but trained with new data

        # create lgb.Dataset for both train and test for lightgbm library use
        train_data = lgb.Dataset(train_x, label=train_y)
        
        # setup lightgbm parameters
        param = {'metric': 'multi_logloss', 'objective': 'multiclass', 'num_class':3}
        param['learning_rate'] = 0.01
        param['max_depth'] = 20
        param['num_leaves'] = 20
        param['min_data_in_leaf'] = 20
        param['min_sum_hessian_in_leaf'] = 1e-3
        param['bagging_fraction'] = 0.9
        param['bagging_freq'] = 5
        param['bagging_seed'] = 3
        param['feature_fraction'] = 0.9
        param['feature_fraction_bynode'] = 0.9
        param['feature_fraction_seed'] = 2
        param['lambda_l1'] = 0.01
        param['lambda_l2'] = 0.01
        param['force_col_wise'] = True
        param['num_threads'] = 4
        if not validation_set_skip:
            print('using validation data to find best iteration')
            num_round = 1000
            bst = lgb.train(param, train_data, num_round, valid_sets=[validation_data], early_stopping_rounds=5,verbose_eval=False)
            best_iteration = bst.best_iteration
        print('using training only with last known good number of iterations')
        num_round = best_iteration
        train_x = full_data[i-training_window_size:i]
        train_y = target[i-training_window_size:i]

        train_x, train_y = oversample.fit_resample(train_x, train_y.target)
        train_data = lgb.Dataset(train_x, label=train_y)
        
        bst = lgb.train(param, train_data, num_round, verbose_eval=False)
        pred = bst.predict(test_x, num_iteration_predict = bst.best_iteration)
        
        predictions.extend(pred)
        truths.extend(test_y)
    prediction_results.insert(2, 'predictions', predictions)
    
    return bst, prediction_results

In [13]:
def plot(standard_data):
    """
    Draw interactive candle stick chart OHLC Volume
    
    Parameters
    ----------
    standard_data : pandas dataframe that contains ['date','high','low','close','open','volume'] columns.
    
    """
    qf = cf.QuantFig(standard_data,legend='bottom')
    qf.add_volume()
    qf.iplot()

In [14]:
# get raw data
raw_data = MS.get('IWM')

In [15]:
raw_data.tail()

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,symbol,exchange,date
2501,82.21,82.8200,82.1783,82.680,38284670.0,72.239514,71.679793,72.117399,71.707443,38284670.0,IWM,ARCX,2011-02-16T00:00:00+0000
2502,82.27,82.4886,81.8300,82.018,42908536.0,71.950451,71.375989,71.539971,71.759778,42908536.0,IWM,ARCX,2011-02-15T00:00:00+0000
2503,82.11,82.6100,82.0500,82.490,36730775.0,72.056342,71.567883,71.951672,71.620218,36730775.0,IWM,ARCX,2011-02-14T00:00:00+0000
2504,80.89,82.1000,80.7900,82.070,38579283.0,71.611496,70.468852,71.585328,70.556077,38579283.0,IWM,ARCX,2011-02-11T00:00:00+0000
2505,80.33,81.2900,80.2400,81.190,47996461.0,70.904975,69.989116,70.817751,70.067618,47996461.0,IWM,ARCX,2011-02-10T00:00:00+0000


In [16]:
# runtimewarnings are produced from ta library but it's nothing to worry about for this project, will need to figure out a way to suppress this warning message.
# data prep steps then
# drop rows with nan
# naturally the last row contains nan since we don't have info from tomorrow. so we are also dropping the last row.
# but in production we will want to keep the last row so we can use it to make prediction
# for modeling and evaluation purposes it's not useful
# after this step the data is ready to use as training dataset
data = raw_data_preprocessing(raw_data)
target = get_target_variable(data)
indicators = get_ta_indicators(data, 'daily')

# concat/merge datasets to create full_data
full_data = indicators

# last step is to remove rows with nan. i.e. first few rows that don't have enough days of data to compute averages etc, and the last row without future data to compute the targer.
full_data, target = remove_nan(full_data, target)

/home/ubuntu/miniconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/home/ubuntu/miniconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



In [17]:
model, prediction_results = train_and_eval(full_data, target)

using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2949, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2724, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2949, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start 

using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2949, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2724, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2949, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start 

using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2718, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2952, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2718, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from sco

using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2733, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2967, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2733, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from sco

[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2751, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2991, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2748, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last kn

...

[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3054, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2907, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3057, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find be

using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3060, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2904, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3063, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start 

using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3066, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2898, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3066, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start 

using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3060, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2904, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3060, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start 

using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3054, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using validation data to find best iteration
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2904, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
using training only with last known good number of iterations
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 3057, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start 

In [18]:
buy_and_hold_result = eval_buy_and_hold(prediction_results, data)
strategy_result = eval_strategy(prediction_results, data)
print('Max Drawdown:')
print('  - Buy and Hold:', str(round(100*min(buy_and_hold_result.draw_down),2))+'%')
print('  - LGB Strategy:', str(round(100*min(strategy_result.draw_down),2))+'%')

#align index number
strategy_result.index = prediction_results.index
buy_and_hold_result.index = prediction_results.index
#insert based on index number
def plot_result(data, benchmark, strategy):
    data = data[:] # making a copy so original data isn't altered.
    data.insert(0,'LGB_Strategy', strategy.ending_balance)
    data.insert(0,'BNH_Strategy', benchmark.ending_balance)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.date[data.BNH_Strategy.notnull()], y=data.BNH_Strategy[data.BNH_Strategy.notnull()], mode='lines', name='Buy and Hold Strategy'))
    fig.add_trace(go.Scatter(x=data.date[data.BNH_Strategy.notnull()], y=data.LGB_Strategy[data.BNH_Strategy.notnull()], mode='lines', name='Lightgbm Strategy'))
    fig.show()
plot_result(data, buy_and_hold_result, strategy_result)

Max Drawdown:
  - Buy and Hold: -41.13%
  - LGB Strategy: -55.72%


In [19]:
pd.concat((data,strategy_result),axis=1).to_csv('./results.csv')

In [20]:
# # holiday info
# min_year = int(min(data['date'])[:4])-2
# max_year = int(max(data['date'])[:4])+2
# min_date = str(min_year)+'-01-01'
# max_date = str(max_year)+'-12-31'
# dates = pd.date_range(min_date,max_date).values
# holidays = holidays.UnitedStates(years=range(min_year,max_year))